**notebook um für gegebene bounding boxes mit dino merkmalsvektoren zu berechnen und zu speichern, damit dies nicht immer im training direkt passieren muss**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!git clone https://github.com/cocodataset/cocoapi

!cd /content/cocoapi/PythonAPI && make

In [ ]:
 !pip install timm
 
 !git clone https://github.com/Moldazien/BA.git

In [ ]:
import os
os.chdir('/content/BA')

In [ ]:
dataset_path  = 'PATH_TO_INAT19'
annotations = '/annotations/meg_train.json'

In [ ]:
import os
import sys
import argparse
import cv2
import random
import colorsys
import requests
from io import BytesIO

import skimage.io
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as pth_transforms
import numpy as np
from PIL import Image

import utils
import vision_transformer as vits

from pycocotools.coco import COCO

In [ ]:
arch = 'vit_base' #'vit_small''vit_base'
patch_size = 8

output_dir = '/content/gdrive/MyDrive/Testing/first_test_dino'
    

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# build model
model = vits.__dict__[arch](patch_size=patch_size, num_classes=0)
for p in model.parameters():
    p.requires_grad = False
model.eval()
model.to(device)


#laden der pretrained weithts für passende modelle   
print("Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.")
url = None
if arch == "vit_small" and patch_size == 16:
    url = "dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth"
elif arch == "vit_small" and patch_size == 8:
    url = "dino_deitsmall8_300ep_pretrain/dino_deitsmall8_300ep_pretrain.pth"  # model used for visualizations in our paper
elif arch == "vit_base" and patch_size == 16:
    url = "dino_vitbase16_pretrain/dino_vitbase16_pretrain.pth"
elif arch == "vit_base" and patch_size == 8:
    url = "dino_vitbase8_pretrain/dino_vitbase8_pretrain.pth"
if url is not None:
    print("Since no pretrained weights have been provided, we load the reference pretrained DINO weights.")
    state_dict = torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/" + url)
    model.load_state_dict(state_dict, strict=True)


In [ ]:
import numpy as np

#funktion um bildausschnitte mit rand zu croppen
def crop_fkt(bbox, img, area, percentile):
  width, height = img.size

  width_edge = bbox[2]*percentile
  height_edge = bbox[3]*percentile

  edge = (width_edge + height_edge)

  tx = max(bbox[0]-edge, 0)
  ty = max(bbox[1]-edge, 0)
  bx = min(bbox[0]+bbox[2]+edge, width)
  by = min(bbox[1]+bbox[3]+edge, height)

  h_box = bx - tx
  w_box = by - ty

  scaling_factor = np.sqrt(area/(h_box*w_box))

  h_scale = int(np.round(h_box*scaling_factor))
  w_scale = int(np.round(w_box*scaling_factor)) 

  return (tx, ty, bx, by), (w_scale, h_scale), edge

In [ ]:
import numpy as np
from torchvision.transforms.functional import InterpolationMode

#funktion um bilder direkt mit dino zu merkmalsvektoren zu verarbeiten

def DINO_fwd(model, image, bbox):
  percentile = 0.3 
  patch_size = 8
  image_size = (480, 480)
  area = 230000 

  box, image_size, edge = crop_fkt(bbox, image, area, percentile)

  crop_img = image.crop(box=box)

  transform = pth_transforms.Compose([
                                      pth_transforms.Resize(200, interpolation=3),
                                      pth_transforms.ToTensor(),
                                      pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                      ])
  crop_img = transform(crop_img)

  w, h = crop_img.shape[1] - crop_img.shape[1] % patch_size, crop_img.shape[2] - crop_img.shape[2] % patch_size
  crop_img = crop_img[:, :w, :h].unsqueeze(0)

  features = model(crop_img.to(device)).clone()
  features = features.detach().to(torch.device('cpu'))

  return features

datenklasse


In [ ]:
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

class NACTI_seg(Dataset):
  def __init__(self, annotations_file, dataset_dir):
    self.annotations_file = annotations_file
    self.coco = COCO(annotations_file)  #for fast reading
    
    
    self.dataset = dataset_dir
    self.img_ids = self.coco.getImgIds()


  def __len__(self):
    return len(self.img_ids)

  #nicht ganz korrekt, gibt RGB bild zurück statt direkten tensor
  def __getitem__(self, idx):
    img_id = self.img_ids[idx]

    img = self.coco.loadImgs(img_id)

    ann_ids = self.coco.getAnnIds(img[0]['id'])
    anns = self.coco.loadAnns(ann_ids)

    img_path = self.dataset + '/images/' + img[0]['file_name']
    image = None
    mask = None
    bbox = None
    with open(img_path, 'rb') as f:
        image = Image.open(f)
        image = image.convert('RGB')
    
    if len(anns) > 0:
      bbox = anns[0]['bbox']
    else:
      bbox = [0,0,img[0]['width'], img[0]['height']]

    features = DINO_fwd(model, image, bbox)


    return features, img[0]['id'], img[0]['file_name']

In [ ]:
%time #zeit messen, weils interessant ist

starting_point = 0

import os

dataset_path = 'PATH_TO_iNaturalist19'
feature_direc = '/features/'


trainset = NACTI_seg('PATH_TO_iNaturalist19_ANNOTATION', dataset_path)

for j in range(trainset.__len__() - starting_point):

  i = j + starting_point

  features, img_id, file_name = trainset.__getitem__(i)

  fileN = file_name.split('/')

  classN = fileN[1]
  direcN = fileN[2]
  imgN = fileN[3].split('.')[0]

  if not os.path.exists(dataset_path + feature_direc + '/' + classN):
    os.mkdir(dataset_path + feature_direc + '/' + classN)
  
  if not os.path.exists(dataset_path + feature_direc + '/' + classN + '/' + direcN):
    os.mkdir(dataset_path + feature_direc + '/' + classN + '/' + direcN)

  if i % 500 == 0:
    drive.flush_and_unmount()

    drive.mount('/content/gdrive')
    print(str(i) + ' von ' + str(trainset.__len__()) + '     ' + str(features.shape))

  feature_path = dataset_path + feature_direc + '/' + classN + '/' + direcN + '/' + imgN + '_feature.pt'
  
  torch.save(features.clone(), feature_path)
  torch.save(mask, mask_path)

# saving in gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
